In [1]:
# allows to import of modules
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
nb_dir
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

triplet loss function https://pytorch.org/docs/stable/nn.html#tripletmarginloss

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
import pandas as pd

In [4]:
import  datatset
from torch.utils.data import DataLoader

In [5]:
# DataLoader

In [6]:
TRAIN_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/train/")
df = pd.read_csv(TRAIN_PATH/"train.csv")
df_train = df[[ "file_name", "category_id"]]

In [7]:
label_counts  = df["category_id"].value_counts()

In [8]:
num_calsses = len(label_counts) + 1

In [9]:
labels_ordered = label_counts.keys().tolist()

In [10]:
num_calsses

32094

In [11]:
ds = datatset.HerbDataSet(df_train, TRAIN_PATH,128,label="category_id")

In [12]:
data_loader = DataLoader(ds, batch_size=32, shuffle=True)

In [21]:
# Tracking

In [22]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter(TRAIN_PATH/"logs/run-2")

In [23]:
import metrics
from train import Training

In [24]:
acc =  metrics.Accuracy()
manager = metrics.MetricManager([acc], writer)

In [25]:
# Train

In [26]:
class Trainer(Training):
    def __init(self,  metrics, loss, optim, data, epochs, model, save_dir):
        super(self).__init__(self,  metrics, loss, optim, data, epochs, model, save_dir)
        
    def train_step(self, batch):
        # data.cuda(), labels.cuda() = batch
        data, labels = batch
        output = self.model(data, labels)
        #     loss = self.loss(preds, labels)
#         self.metrics.update(preds, labels, self.step)
        if not self.metrics.writer:
            self.metrics.writer.add_scalar("loss", loss.item(), self.step)
            # _logger.debug(f'Loss: {loss.item()}')

        self.optim.zero_grad()  # zero gradients
        output.loss.backward()  # calculate gradients
        self.optim.step()  # updated weights


In [27]:
criterion = nn.CrossEntropyLoss()
# criterion = nn.AdaptiveLogSoftmaxWithLoss(num_calsses, num_calsses, [50, 500, 5000])

In [28]:
optimizer = torch.optim.Adam(model.parameters())

In [29]:
model_path=TRAIN_PATH/"models/"

In [30]:
trainer = Trainer(manager, criterion, optimizer, data_loader, 3, model, model_path )

In [31]:
trainer.train_loop()

Epoch 0/3


RuntimeError: CUDA out of memory. Tried to allocate 984.00 MiB (GPU 0; 5.81 GiB total capacity; 3.81 GiB already allocated; 336.94 MiB free; 175.82 MiB cached)

In [2]:
# prints currently alive Tensors and Variables
import torch
import gc
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size())
    except:
        pass

/home/shaun/personal/.pytorch-env/lib/python3.6/site-packages/torch/distributed/distributed_c10d.py:100: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
